In [1]:
import os
os.chdir('../')

import warnings
warnings.filterwarnings('ignore')

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os

!nvidia-smi
os.environ["CUDA_VISIBLE_DEVICES"]="4"

Thu Jul 20 18:41:13 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.125.06   Driver Version: 525.125.06   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A5000    Off  | 00000000:1B:00.0 Off |                    0 |
| 30%   25C    P8    13W / 230W |   2522MiB / 23028MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A5000    Off  | 00000000:1C:00.0 Off |                  Off |
| 30%   

### Hyperparams

In [3]:
n_mels = 768
n_outputs = 61
n_frames = 400

### Model

In [4]:
from model.model_transformer_reg import Model
from utils.util import *
from tensorboardX import SummaryWriter

step = 0
device = 'cuda:0'

# Model
model = Model(in_dim=n_mels, h_dim=512, out_dim=n_outputs)
model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
print('done')

done


In [5]:
# warm start
checkpoint = torch.load('/data/scpark/save/lips/train06.23-1/save_120000', map_location=torch.device('cpu'))
model_state_dict = model.state_dict()

for key in checkpoint['model_state_dict']:
    if key in model_state_dict.keys():
        if checkpoint['model_state_dict'][key].shape == model_state_dict[key].shape:
            model_state_dict[key] = checkpoint['model_state_dict'][key]
            print(key)
model.load_state_dict(model_state_dict, strict=True)
print('warm start')

prenet.bias
encoder.attn_layers.0.emb_rel_k
encoder.attn_layers.0.emb_rel_v
encoder.attn_layers.0.conv_q.weight
encoder.attn_layers.0.conv_q.bias
encoder.attn_layers.0.conv_k.weight
encoder.attn_layers.0.conv_k.bias
encoder.attn_layers.0.conv_v.weight
encoder.attn_layers.0.conv_v.bias
encoder.attn_layers.0.conv_o.weight
encoder.attn_layers.0.conv_o.bias
encoder.attn_layers.1.emb_rel_k
encoder.attn_layers.1.emb_rel_v
encoder.attn_layers.1.conv_q.weight
encoder.attn_layers.1.conv_q.bias
encoder.attn_layers.1.conv_k.weight
encoder.attn_layers.1.conv_k.bias
encoder.attn_layers.1.conv_v.weight
encoder.attn_layers.1.conv_v.bias
encoder.attn_layers.1.conv_o.weight
encoder.attn_layers.1.conv_o.bias
encoder.attn_layers.2.emb_rel_k
encoder.attn_layers.2.emb_rel_v
encoder.attn_layers.2.conv_q.weight
encoder.attn_layers.2.conv_q.bias
encoder.attn_layers.2.conv_k.weight
encoder.attn_layers.2.conv_k.bias
encoder.attn_layers.2.conv_v.weight
encoder.attn_layers.2.conv_v.bias
encoder.attn_layers.2.conv

### Load

In [6]:
save_dir = '/data/scpark/save/lips/train07.20-1/'
!mkdir -p $save_dir
!ls -lt $save_dir

writer = SummaryWriter(save_dir)

if False:
    step, model, _, optimizer = load(save_dir, 133000, model, None, optimizer)

total 521832
-rw-rw-r-- 1 scpark scpark 534355839  7월 20 18:40 save_0
-rw-rw-r-- 1 scpark scpark         0  7월 20 18:39 events.out.tfevents.1689845967.GPUSVR01
-rw-rw-r-- 1 scpark scpark         0  7월 20 18:29 events.out.tfevents.1689845389.GPUSVR01
-rw-rw-r-- 1 scpark scpark         0  7월 20 18:27 events.out.tfevents.1689845277.GPUSVR01
-rw-rw-r-- 1 scpark scpark         0  7월 20 18:26 events.out.tfevents.1689845180.GPUSVR01
-rw-rw-r-- 1 scpark scpark         0  7월 20 18:23 events.out.tfevents.1689845034.GPUSVR01
-rw-rw-r-- 1 scpark scpark         0  7월 20 18:14 events.out.tfevents.1689844454.GPUSVR01
-rw-rw-r-- 1 scpark scpark         0  7월 20 18:14 events.out.tfevents.1689844452.GPUSVR01


### Dataset

In [7]:
import os
from data.arkit_dataset import LipsDataset, CombinedDataset, CombinedCollate

root_dir = '/data/speech/digital_human/preprocessed/'
files = sorted([os.path.join(root_dir, file) for file in os.listdir(root_dir)])
print(len(files))

train_datasets = []
test_datasets = []

for file in files:
    print(file)
    dataset = LipsDataset(file, n_mels, n_frames)
    if '_10_' in file:
        test_datasets.append(dataset)
    else:
        train_datasets.append(dataset)
print(len(train_datasets), len(test_datasets))

39
/data/speech/digital_human/preprocessed/MH_ARKit_001_1_iPhone_raw.npy
/data/speech/digital_human/preprocessed/MH_ARKit_001_2_iPhone_raw.npy
/data/speech/digital_human/preprocessed/MH_ARKit_001_3_iPhone_raw.npy
/data/speech/digital_human/preprocessed/MH_ARKit_001_4_iPhone_raw.npy
/data/speech/digital_human/preprocessed/MH_ARKit_001_5_iPhone_raw.npy
/data/speech/digital_human/preprocessed/MH_ARKit_001_6_iPhone_raw.npy
/data/speech/digital_human/preprocessed/MH_ARKit_001_7_iPhone_raw.npy
/data/speech/digital_human/preprocessed/MH_ARKit_001_8_iPhone_raw.npy
/data/speech/digital_human/preprocessed/MH_ARKit_001_9_iPhone_raw.npy
/data/speech/digital_human/preprocessed/MH_ARKit_002_10_iPhone_raw.npy
/data/speech/digital_human/preprocessed/MH_ARKit_002_1_iPhone_raw.npy
/data/speech/digital_human/preprocessed/MH_ARKit_002_2_iPhone_raw.npy
/data/speech/digital_human/preprocessed/MH_ARKit_002_3_iPhone_raw.npy
/data/speech/digital_human/preprocessed/MH_ARKit_002_4_iPhone_raw.npy
/data/speech/dig

In [8]:
train_loader = torch.utils.data.DataLoader(CombinedDataset(train_datasets), 
                                           num_workers=16, shuffle=True, batch_size=32, collate_fn=CombinedCollate())
test_loader = torch.utils.data.DataLoader(CombinedDataset(test_datasets), 
                                          num_workers=10, shuffle=True, batch_size=10, collate_fn=CombinedCollate())
print('done')

done


In [9]:
import fairseq
from torchaudio.transforms import Resample

resample = Resample(24000, 16000)

ckpt_path = "/Storage/speech/pretrained/contentvec/checkpoint_best_legacy_500.pt"
hubert, cfg, task = fairseq.checkpoint_utils.load_model_ensemble_and_task([ckpt_path])
hubert = hubert[0]
hubert = hubert.to(device)
hubert.eval()

def get_hubert_feature(wav):
    with torch.no_grad():
        # (b, t, c)
        wav = resample(torch.Tensor(wav)).to(device)
        feature = hubert.extract_features(wav, output_layer=12)[0]
        return feature.transpose(1, 2)
print('done')

2023-07-20 18:41:42 | INFO | fairseq.tasks.hubert_pretraining | current directory is /home/scpark/projects/wav2face
2023-07-20 18:41:42 | INFO | fairseq.tasks.hubert_pretraining | HubertPretrainingTask Config {'_name': 'hubert_pretraining', 'data': 'metadata', 'fine_tuning': False, 'labels': ['km'], 'label_dir': 'label', 'label_rate': 50.0, 'sample_rate': 16000, 'normalize': False, 'enable_padding': False, 'max_keep_size': None, 'max_sample_size': 250000, 'min_sample_size': 32000, 'single_target': False, 'random_crop': True, 'pad_audio': False}
2023-07-20 18:41:42 | INFO | fairseq.models.hubert.hubert | HubertModel Config: {'_name': 'hubert', 'label_rate': 50.0, 'extractor_mode': default, 'encoder_layers': 12, 'encoder_embed_dim': 768, 'encoder_ffn_embed_dim': 3072, 'encoder_attention_heads': 12, 'activation_fn': gelu, 'layer_type': transformer, 'dropout': 0.1, 'attention_dropout': 0.1, 'activation_dropout': 0.0, 'encoder_layerdrop': 0.05, 'dropout_input': 0.1, 'dropout_features': 0.1,

done


In [ ]:
from IPython import display
import librosa.display
import matplotlib.pyplot as plt

isnan = False
while True:
    if isnan:
        break
    for batch in train_loader:
        targets = torch.Tensor(batch['blend']).transpose(1, 2).to(device)
        inputs = get_hubert_feature(batch['wav'])
        inputs = F.interpolate(inputs, size=(targets.shape[2]), mode='linear')
        
        model.train()
        model.zero_grad()
        outputs = model(inputs, targets)
        
        print(step)
        loss = 0
        for key in outputs.keys():
            if 'loss' in key:
                loss += outputs[key]
                print(key, outputs[key].item())
        if torch.isnan(loss):
            isnan = True
            break
        loss.backward()
        optimizer.step()
        print(step, loss.item())
        
        if step % 1000 == 0:
            writer.add_scalar('train_loss', loss.item(), step)
        
        if step % 1000 == 0:
            display.clear_output()
            
            losses = []
            for i, batch in enumerate(test_loader):
                if i >= 10:
                    break
                    
                targets = torch.Tensor(batch['blend']).transpose(1, 2).to(device)
                inputs = get_hubert_feature(batch['wav'])
                inputs = F.interpolate(inputs, size=(targets.shape[2]), mode='linear')

                model.eval()
                with torch.no_grad():
                    outputs = model(inputs, targets)
                    
                loss = 0
                for key in outputs.keys():
                    if 'loss' in key:
                        loss += outputs[key]
                print('test :', i, loss.item())
                losses.append(loss)        
            
            test_loss = torch.stack(losses).mean().item()
            print('test_loss :', test_loss)
            writer.add_scalar('test_loss', test_loss, step)
            
#             plt.figure(figsize=[18, 4])
#             librosa.display.specshow(targets[0].data.cpu().numpy(), cmap='magma')
#             plt.show()

#             plt.figure(figsize=[18, 4])
#             librosa.display.specshow(outputs['y_pred'][0].data.cpu().numpy(), cmap='magma')
#             plt.show()
            
#             for i in [20, 37]:
#                 plt.figure(figsize=[18, 2])
#                 plt.title(str(i))
#                 plt.plot(targets[0].data.cpu().numpy()[i])
#                 plt.plot(outputs['y_pred'][0].data.cpu().numpy()[i])
#                 plt.show()

        if step % 1000 == 0:
            save(save_dir, step, model, None, optimizer)
    
        step += 1

test : 0 0.05214358866214752
test : 1 0.057785555720329285
test : 2 0.05865137651562691
test : 3 0.04913139343261719
test : 4 0.06084894761443138
test : 5 0.05780322849750519
test : 6 0.06041058525443077
test : 7 0.05593559890985489
test : 8 0.05820368975400925
test : 9 0.05801958590745926
test_loss : 0.05689335614442825
saved /data/scpark/save/lips/train07.20-1/save_0
1
loss 0.05239885300397873
1 0.05239885300397873
2
loss 0.04586300253868103
2 0.04586300253868103
3
loss 0.047861773520708084
3 0.047861773520708084
4
loss 0.051491837948560715
4 0.051491837948560715
5
loss 0.04675668105483055
5 0.04675668105483055
6
loss 0.05098067596554756
6 0.05098067596554756
7
loss 0.052576612681150436
7 0.052576612681150436
8
loss 0.05175600200891495
8 0.05175600200891495
9
loss 0.04775285720825195
9 0.04775285720825195
10
loss 0.04956114664673805
10 0.04956114664673805
11
loss 0.04847735911607742
11 0.04847735911607742
12
loss 0.04936465993523598
12 0.04936465993523598
13
loss 0.045202407985925674

In [ ]:
save(save_dir, step, model, None, optimizer)